In [ ]:
!pip install langchain unstructured tiktoken pinecone-client pypdf  langchain_pinecone langchain-community PyPDF2 sentence_transformers groq langchain-groq --quiet --

In [ ]:
import os
import pandas as pd
import numpy as np
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pathlib
import textwrap
from groq import Groq
from IPython.display import display, Markdown
import os
from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq

In [ ]:
pinecone.__version__

'5.0.0'

In [ ]:
GROQ_API_KEY = "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"
PINECONE_API_KEY = "1307be7e-dd9f-41e4-af3d-70166f43ba98"
index_name = "innovent1"

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        pod_type='p1'
    )

index = pc.Index(index_name)

stats = index.describe_index_stats()

print(f"Number of vectors in the index '{index_name}': {stats['total_vector_count']}")

Number of vectors in the index 'innovent1': 981


In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query_text = "Knurling operation"

def retrieve_query(query_embedding, k=1):
    matching_results = index.query(vector=query_embedding, top_k=k, include_metadata=True)
    return matching_results
query_embedding = embedding_model.encode(query_text)
query_embedding = query_embedding.tolist() if hasattr(query_embedding, 'tolist') else query_embedding
results = retrieve_query(query_embedding)

print("Retrieved Results:")
for result in results['matches']:
    print(result)

Retrieved Results:
{'id': '364',
 'metadata': {'chunk_id': 364.0,
              'text': 'Fundamentals\t\r'
                      ' \xa0of\t\r'
                      ' \xa0CNC\t\r'
                      ' \xa0Machining\t\r'
                      ' \xa0\t\r'
                      ' \xa0Lesson\t\r'
                      ' \xa06\t\r'
                      ' \xa0CNC\t\r'
                      ' \xa0Operation\t\r'
                      ' \xa0\n'
                      '6-\xad‐18\t\r'
                      ' \xa0\t\r'
                      ' \xa0\t\r'
                      ' \xa0\t\r'
                      ' \xa0Copyright\t\r'
                      ' \xa02014\t\r'
                      ' \xa0Autodesk,\t\r'
                      ' \xa0Inc.\t\r'
                      ' \xa05\t\r'
                      ' \xa0\t\r'
                      ' \xa0Handle\t\r'
                      ' \xa0Jog:\t\r'
                      ' \xa0Select\t\r'
                      ' \xa0Jog\t\r'
                      ' \xa0In

In [ ]:
text = results['matches'][0]['metadata']['text']

In [ ]:
cleaned_text = text.replace('\t', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('§\uf0a7', '').replace('\xad‐', '-')
cleaned_text = ' '.join(cleaned_text.split())

print(cleaned_text)

Fundamentals of CNC Machining Lesson 6 CNC Operation 6--18 Copyright 2014 Autodesk, Inc. 5 Handle Jog: Select Jog Increment: .01 6 Jog Handle: As Needed Select jog direction and use handle as required to place edge finder stylus alongside the left part edge. 7 Jog Increment: .001 Move edge finder slowly until it just trips off center as shown below. This places the center of the spindle exactly .100 from the part edge.


In [ ]:
ai_role = "You are a AI assistant that helps people resolve their error in industrial machinary the user has give the error or problem they are facing and the relevent solution is given convert this entire query into a step by step guide for solving the error"
user_input = query_text
query_to_gen = ai_role + user_input + cleaned_text
query_to_gen

'You are a AI assistant that helps people resolve their error in industrial machinary the user has give the error or problem they are facing and the relevent solution is given convert this entire query into a step by step guide for solving the errorKnurling operationFundamentals of CNC Machining Lesson 6 CNC Operation 6--18 Copyright 2014 Autodesk, Inc. 5 Handle Jog: Select Jog Increment: .01 6 Jog Handle: As Needed Select jog direction and use handle as required to place edge finder stylus alongside the left part edge. 7 Jog Increment: .001 Move edge finder slowly until it just trips off center as shown below. This places the center of the spindle exactly .100 from the part edge.'

In [ ]:
client = Groq(api_key=GROQ_API_KEY)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query_to_gen,
        }
    ],
    model="llama3-70b-8192",
)
print(chat_completion.choices[0].message.content)

**Step-by-Step Guide to Resolving Knurling Operation Error**

**Problem Statement:** Error during Knurling operation in CNC Machining.

**Error Description:** The machine is not accurately positioning the edge finder alongside the part edge, resulting in an incorrect knurling operation.

**Solution:**

**Step 1: Handle Jog**

* Select the Jog Increment as `.01`
* Use the Jog Handle as needed to place the edge finder stylus alongside the left part edge.

**Step 2: Jog Direction and Edge Finder Positioning**

* Select the jog direction to move the edge finder slowly towards the part edge.
* Use the Jog Handle to carefully position the edge finder alongside the part edge.

**Step 3: Fine-Tune Edge Finder Position**

* Change the Jog Increment to `.001`
* Move the edge finder slowly until it just trips off center.
* This will place the center of the spindle exactly `.100` from the part edge, ensuring accurate knurling operation.

**Verification:**

* Check the edge finder position to ensur

In [ ]:
ai_role = "You are a AI assistant that helps people resolve their error in industrial machinary the user has give the error or problem they are facing and the relevent solution is given convert this entire query into a step by step guide for solving the error"

groq_chat = ChatGroq(
            groq_api_key=GROQ_API_KEY,
            model_name="llama3-70b-8192"
    )

print("Hello! I'm your here to solve any error you face while handling a Machine. I can help answer your questions and solve errors!")

system_prompt = 'You are a friendly conversational chatbot'
conversational_memory_length = 5

memory = ConversationBufferWindowMemory(k=conversational_memory_length, memory_key="chat_history", return_messages=True)
while True:
        user_question = input("Ask a question: (To end chat enter : END) : ")
        if user_question == "END":
          break;
        if user_question:
            results = retrieve_query(user_question)
            text = results['matches'][0]['metadata']['text']
            cleaned_text = text.replace('\t', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('§\uf0a7', '').replace('\xad‐', '-')
            cleaned_text = ' '.join(cleaned_text.split())
            user_question = ai_role + user_question + cleaned_text
            prompt = ChatPromptTemplate.from_messages(
                [
                    SystemMessage(
                        content=system_prompt
                    ),

                    MessagesPlaceholder(
                        variable_name="chat_history"
                    ),

                    HumanMessagePromptTemplate.from_template(
                        "{human_input}"
                    ),
                ]
            )
            conversation = LLMChain(
                llm=groq_chat,
                prompt=prompt,
                verbose=False,
                memory=memory,
            )
            response = conversation.predict(human_input=user_question)
            print("Chatbot:", response)

Hello! I'm your friendly Groq chatbot. I can help answer your questions, provide information, or just chat. I'm also super fast! Let's start our conversation!
Ask a question: (To end chat enter : END)END


# PDF from Scratch

In [ ]:
!pip install -q langchain
!pip install -q unstructured
!pip install -q tiktoken
!pip install -q pinecone-client
!pip install -q pypdf
!pip install -q langchain_pinecone
!pip install -q langchain-community
!pip install -q PyPDF2
!pip install -q sentence_transformers
!pip install -q groq
!pip install -q langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 359.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 1.9 MB/s eta 0:00:00
  

In [ ]:
import os
import pandas as pd
import numpy as np
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pathlib
import textwrap
from groq import Groq
from IPython.display import display, Markdown
import os
from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq

In [ ]:
GROQ_API_KEY = "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"
PINECONE_API_KEY = "d40666d4-1716-4ca5-8ad1-2e26c20c42f9"
index_name = "innovent"

In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def read_doc(file_path):
    file_loader = PyPDFLoader(file_path)
    documents = file_loader.load()
    return documents
def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(docs)
    return chunks

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        pod_type='p1'
    )

index = pc.Index(index_name)


In [ ]:
def retrieve_query(query_embedding, k=1):
    matching_results = index.query(vector=query_embedding, top_k=k, include_metadata=True)
    return matching_results

In [ ]:
# Taking pdf path as input and uplading input to pinecone directly and directly playing the chatbot
def upload_pdf_for_troubleshooting(pdf_path):

  doc=read_doc(pdf_path)
  print("Length of document", len(doc))
  documents=chunk_data(docs=doc)
  print("Length of documents", len(documents))
  texts = [chunk.page_content for chunk in documents]
  print("Length of texts", len(texts))

  vectors = [embedding_model.encode(text) for text in texts]
  for idx, (embedding, text) in enumerate(zip(vectors, texts)):
    metadata = {"text": text, "chunk_id": idx}
    index.upsert(vectors=[(str(idx), embedding.tolist(), metadata)])
  print("Your Embeddings are Uploaded to Pinecone Successfully")
  ai_role = "You are a AI assistant that helps people resolve their error in industrial machinary the user has give the error or problem they are facing and the relevent solution is given convert this entire query into a step by step guide for solving the error"

  groq_chat = ChatGroq(
              groq_api_key=GROQ_API_KEY,
              model_name="llama3-70b-8192"
      )

  print("Hello! I'm your here to solve any error you face while handling a Machine. I can help answer your questions and solve errors!")

  system_prompt = 'You are a friendly conversational chatbot'
  conversational_memory_length = 5

  memory = ConversationBufferWindowMemory(k=conversational_memory_length, memory_key="chat_history", return_messages=True)
  while True:
          user_question = input("Ask a question: (To end chat enter : END) : ")
          if user_question == "END":
            break;
          if user_question:
              query_embedding = embedding_model.encode(user_question)
              query_embedding = query_embedding.tolist() if hasattr(query_embedding, 'tolist') else query_embedding
              results = retrieve_query(query_embedding)

              text = results['matches'][0]['metadata']['text']
              cleaned_text = text.replace('\t', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('§\uf0a7', '').replace('\xad‐', '-')
              cleaned_text = ' '.join(cleaned_text.split())
              user_question = ai_role + user_question + cleaned_text
              prompt = ChatPromptTemplate.from_messages(
                  [
                      SystemMessage(
                          content=system_prompt
                      ),

                      MessagesPlaceholder(
                          variable_name="chat_history"
                      ),

                      HumanMessagePromptTemplate.from_template(
                          "{human_input}"
                      ),
                  ]
              )
              conversation = LLMChain(
                  llm=groq_chat,
                  prompt=prompt,
                  verbose=False,
                  memory=memory,
              )
              response = conversation.predict(human_input=user_question)
              print("Chatbot:", response)

In [ ]:
upload_pdf_for_troubleshooting('/content/CNC.pdf')

Length of document 256
Length of documents 981
Length of texts 981
Your Embeddings are Uploaded to Pinecone Successfully
Hello! I'm your here to solve any error you face while handling a Machine. I can help answer your questions and solve errors!
Ask a question: (To end chat enter : END) : I have problem in Knurling Operation


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Chatbot: Here is a step-by-step guide to resolve the error in Knurling Operation:

**Error:** Issue with Knurling Operation in CNC Machining

**Step-by-Step Solution:**

**Step 1: Handle Jog**

* Select the Jog Increment: .01
* Use the jog handle to move the spindle to the desired position

**Step 2: Select Jog Direction**

* Choose the correct jog direction to place the edge finder stylus alongside the left part edge

**Step 3: Move Edge Finder**

* Jog the handle as needed to move the edge finder slowly until it just trips off center
* This will place the center of the spindle exactly .100 from the part edge

**Step 4: Verify Position**

* Check that the center of the spindle is precisely .100 from the part edge
* If not, adjust the jog handle and edge finder as needed to achieve the correct position

**Step 5: Continue with Knurling Operation**

* Once the correct position is achieved, proceed with the knurling operation as usual

By following these steps, you should be able to reso

In [ ]:
doc=read_doc('/content/CNC.pdf')
print(len(doc))

documents=chunk_data(docs=doc)
len(documents)

In [ ]:
texts = [chunk.page_content for chunk in documents]
print(len(texts))
vectors=embedding_model.encode(texts)
len(vectors)
vectors

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        pod_type='p1'
    )

index = pc.Index(index_name)

vectors = [embedding_model.encode(text) for text in texts]

for idx, (embedding, text) in enumerate(zip(vectors, texts)):
    metadata = {"text": text, "chunk_id": idx}
    index.upsert(vectors=[(str(idx), embedding.tolist(), metadata)])

In [ ]:
query_text = "Knurling operation"

def retrieve_query(query_embedding, k=1):
    matching_results = index.query(vector=query_embedding, top_k=k, include_metadata=True)
    return matching_results
query_embedding = embedding_model.encode(query_text)
query_embedding = query_embedding.tolist() if hasattr(query_embedding, 'tolist') else query_embedding
results = retrieve_query(query_embedding)

print("Retrieved Results:")
for result in results['matches']:
    print(result)

In [ ]:
ai_role = "You are a AI assistant that helps people resolve their error in industrial machinary the user has give the error or problem they are facing and the relevent solution is given convert this entire query into a step by step guide for solving the error"

groq_chat = ChatGroq(
            groq_api_key=GROQ_API_KEY,
            model_name="llama3-70b-8192"
    )

print("Hello! I'm your here to solve any error you face while handling a Machine. I can help answer your questions and solve errors!")

system_prompt = 'You are a friendly conversational chatbot'
conversational_memory_length = 5

memory = ConversationBufferWindowMemory(k=conversational_memory_length, memory_key="chat_history", return_messages=True)
while True:
        user_question = input("Ask a question: (To end chat enter : END) : ")
        if user_question == "END":
          break;
        if user_question:
            query_embedding = embedding_model.encode(user_question)
            query_embedding = query_embedding.tolist() if hasattr(query_embedding, 'tolist') else query_embedding
            results = retrieve_query(query_embedding)

            text = results['matches'][0]['metadata']['text']
            cleaned_text = text.replace('\t', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('§\uf0a7', '').replace('\xad‐', '-')
            cleaned_text = ' '.join(cleaned_text.split())
            user_question = ai_role + user_question + cleaned_text
            prompt = ChatPromptTemplate.from_messages(
                [
                    SystemMessage(
                        content=system_prompt
                    ),

                    MessagesPlaceholder(
                        variable_name="chat_history"
                    ),

                    HumanMessagePromptTemplate.from_template(
                        "{human_input}"
                    ),
                ]
            )
            conversation = LLMChain(
                llm=groq_chat,
                prompt=prompt,
                verbose=False,
                memory=memory,
            )
            response = conversation.predict(human_input=user_question)
            print("Chatbot:", response)